In [ ]:
import pandas as pd
from Scripts.dirty_accuracy import injection

In [ ]:
ecoli = pd.read_csv("datasets/ecoli.csv.csv", sep=',')
ecoli_list = injection(df_pandas=ecoli, seed=10, name='ecoli', name_class='class')
ecoli_list.append(ecoli)
ecoli_list.reverse()